### Pandas 数据处理

In [540]:
import pandas as pd
import numpy as np

In [541]:
local = pd.read_csv('local_nestle_Jan.csv', encoding= 'utf-8')
ti = pd.read_csv('ti_nestle_Jan.csv',encoding= 'utf-8')
mapping = pd.read_excel('store_url_mapping.xlsx')

ti['username'] = np.NAN
ti['variant_1'] = np.NAN
ti['variant_2'] = np.NAN
ti['review_id'] = np.NAN

local.rename(columns={'store':'Store','retailer_product_code':'rpc','harvest_product_description':'product_description', 'creation_date':'review_date'},inplace=True)
ti.rename(columns={'store':'Store','retailer_product_code':'rpc','harvest_product_description':'product_description'}, inplace=True)

mapping['rpc'] = mapping['rpc'].astype(str)
local['rpc'] = local['rpc'].astype(str)
ti['rpc'] = ti['rpc'].astype(str)

mapping.head()

,Store,Sub_store,Store_url,Brand,url,rpc
0,JD,雀巢母婴京东自营旗舰店,https://mall.jd.com/index-1000003112.html,NAN HA,https://item.jd.com/4396232.html,4396232
1,JD,雀巢母婴京东自营旗舰店,https://mall.jd.com/index-1000003112.html,NAN HA,https://item.jd.com/4396142.html,4396142
2,JD,雀巢母婴京东自营旗舰店,https://mall.jd.com/index-1000003112.html,NAN HA,https://item.jd.com/3849865.html,3849865
3,JD,雀巢母婴京东自营旗舰店,https://mall.jd.com/index-1000003112.html,NAN HA,https://item.jd.com/4209173.html,4209173
4,JD,雀巢母婴京东自营旗舰店,https://mall.jd.com/index-1000003112.html,NAN HA,https://item.jd.com/5285870.html,5285870


In [542]:
mapping['Store'].value_counts()

Tmall     205
JD        126
Suning     33
Name: Store, dtype: int64

In [543]:
local_mapped = local.loc[local['rpc'].isin(mapping['rpc']) * local['Store'].isin(mapping['Store'])]
ti_mapped = ti.loc[ti['rpc'].isin(mapping['rpc']) * ti['Store'].isin(mapping['Store'])]
print(local_mapped.shape, ti_mapped.shape)

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


(13755, 12) (5838, 17)


In [544]:
ti_mapped['Store'].value_counts()

Suning    5838
Name: Store, dtype: int64

In [545]:
local_mapped_merge = pd.merge(local_mapped, mapping, on = ['rpc'])
ti_mapped_merge = pd.merge(ti_mapped, mapping, on = ['rpc'])
local_mapped_merge.shape, ti_mapped_merge.shape


((13755, 17), (5838, 22))

In [546]:
clm = ['Store_x','Store_y','Sub_store','Store_url','product_description','Brand','rpc','review_id',
       'username','review_rating','variant_1','variant_2','url_x','url_y',
       'review_date',
'review_text']
clm

['Store_x',
 'Store_y',
 'Sub_store',
 'Store_url',
 'product_description',
 'Brand',
 'rpc',
 'review_id',
 'username',
 'review_rating',
 'variant_1',
 'variant_2',
 'url_x',
 'url_y',
 'review_date',
 'review_text']

In [547]:
local_mapped_merge_out = local_mapped_merge[clm]
ti_mapped_merge_out = ti_mapped_merge[clm]

In [548]:
ti_mapped_merge_out.columns == local_mapped_merge_out.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [549]:
nestle_jan_raw_review = pd.concat([ti_mapped_merge_out, local_mapped_merge_out])
nestle_jan_raw_review.shape

(19593, 16)

In [550]:
def fix_text(row):
    review_text = str(row["review_text"]).strip()
    review_text = review_text.replace("\\", "")
    review_text = review_text.replace("\n", "")
    review_text = review_text.replace("\t", "")
    review_text = review_text.replace("\r", "")
    review_text = review_text.replace("\\", "")
    review_text = review_text.replace("\"", "")
    review_text = review_text.replace("\'", "")
    review_text = review_text.replace(",", "，")
    review_text = review_text.replace("*", "")
    review_text = review_text.replace("&gt;", "")
    review_text = review_text.replace("&lt;", "")
    review_text = review_text.replace("&hellip;", "")
    review_text = review_text.replace("&ldquo;", "")
    review_text = review_text.replace("&rdquo;", "")
    review_text = review_text.replace("&nbsp;", "")
    review_text = review_text.replace("&sub;", "")
    review_text = review_text.replace("&omega;", "")
    review_text = review_text.replace("&acute;", "")
    review_text = review_text.replace("&cap;", "")
    review_text = review_text.replace("&Sigma;", "")
    review_text = review_text.replace("&yuml;", "")
    review_text = review_text.replace("&zwnj;", "")
    review_text = review_text.replace(";", "；")
    return review_text

In [551]:
nestle_jan_raw_review['review_text'] = nestle_jan_raw_review.apply(fix_text,axis=1)

In [552]:
nestle_jan_raw_review.columns

Index(['Store_x', 'Store_y', 'Sub_store', 'Store_url', 'product_description',
       'Brand', 'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url_x', 'url_y', 'review_date', 'review_text'],
      dtype='object')

In [553]:
nestle_jan_raw_review.shape

(19593, 16)

In [554]:
#删除其中的'系统默认好评'
nestle_jan_raw_review= nestle_jan_raw_review[True - nestle_jan_raw_review['review_text'].str.contains('系统默认好评')]
nestle_jan_raw_review.shape

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


(18055, 16)

In [555]:
#del nestle_jan_raw_review['Store_y']
#del nestle_jan_raw_review['url_y']
nestle_jan_raw_review.rename(columns={'Store_x':'Store','url_x':'url'}, inplace=True)

nestle_jan_raw_review.columns


Index(['Store', 'Store_y', 'Sub_store', 'Store_url', 'product_description',
       'Brand', 'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url', 'url_y', 'review_date', 'review_text'],
      dtype='object')

In [556]:
nestle_feb_1 = nestle_jan_raw_review[nestle_jan_raw_review['review_date'].str.contains('2018-02-')]
nestle_feb_1.to_csv('nestle_feb_1.csv', index= False, encoding= 'utf-8')

In [557]:
nestle_jan_raw_review = nestle_jan_raw_review[True - nestle_jan_raw_review['review_date'].str.contains('2018-02-')]
nestle_jan_raw_review['Brand'].value_counts()

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


Gerber    8931
NAN       5815
NAN HA    2099
MOM         28
Name: Brand, dtype: int64

In [486]:
# 写出文件(已经将local和ti两部分数据源合并)

nestle_jan_raw_review.to_csv('nestle_jan_raw_review.csv', index=False, encoding='utf-8')

### Review of Feb.


In [558]:
local_feb = pd.read_csv('local_nestle_feb.xlsx.csv', encoding='utf-8')
ti_feb = pd.read_csv('ti_nestle_feb.csv', encoding= 'utf -8')

In [559]:
local_feb.shape, ti_feb.shape

((33449, 15), (23113, 13))

In [560]:
nestle_feb_1.shape

(1182, 16)

In [561]:

ti_feb['username'] = np.NAN
ti_feb['variant_1'] = np.NAN
ti_feb['variant_2'] = np.NAN
ti_feb['review_id'] = np.NAN

local_feb.rename(columns={'store':'Store','retailer_product_code':'rpc','title':'product_description', 'creation_date':'review_date'},inplace=True)
ti_feb.rename(columns={'store':'Store','retailer_product_code':'rpc','harvest_product_description':'product_description'}, inplace=True)

mapping['rpc'] = mapping['rpc'].astype(str)
local_feb['rpc'] = local_feb['rpc'].astype(str)
ti_feb['rpc'] = ti_feb['rpc'].astype(str)


In [562]:
local_feb_mapped = local_feb.loc[local_feb['rpc'].isin(mapping['rpc'])]
ti_feb_mapped = ti_feb.loc[ti_feb['rpc'].isin(mapping['rpc']) * ti_feb['Store'].isin(mapping['Store'])]
local_feb_mapped.shape, ti_feb_mapped.shape

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


((19410, 15), (2364, 17))

In [563]:
local_feb_mapped_merge = pd.merge(local_feb_mapped, mapping, on = ['rpc'])
ti_feb_mapped_merge = pd.merge(ti_feb_mapped, mapping, on = ['rpc'])
local_feb_mapped_merge.shape, ti_feb_mapped_merge.shape


((19410, 20), (2364, 22))

In [564]:
local_feb_mapped_merge.columns

Index(['Store_x', 'sub_store', 'url_x', 'rpc', 'product_description',
       'review_id', 'review_date', 'username', 'review_text', 'review_rating',
       'variant_1', 'variant_2', 'device', 'job_id', 'download_time',
       'Store_y', 'Sub_store', 'Store_url', 'Brand', 'url_y'],
      dtype='object')

In [565]:
local_feb_mapped_merge_out = local_feb_mapped_merge[clm]
ti_feb_mapped_merge_out = ti_feb_mapped_merge[clm]
local_feb_mapped_merge_out.columns

Index(['Store_x', 'Store_y', 'Sub_store', 'Store_url', 'product_description',
       'Brand', 'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url_x', 'url_y', 'review_date', 'review_text'],
      dtype='object')

In [566]:
nestle_feb_1.rename(columns={'Store':'Store_x', 'url':'url_x'}, inplace= True)
nestle_feb_1.columns

Index(['Store_x', 'Store_y', 'Sub_store', 'Store_url', 'product_description',
       'Brand', 'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url_x', 'url_y', 'review_date', 'review_text'],
      dtype='object')

In [567]:
nestle_feb_raw_review = pd.concat([ti_feb_mapped_merge_out, local_feb_mapped_merge_out,nestle_feb_1])
nestle_feb_raw_review.columns

Index(['Store_x', 'Store_y', 'Sub_store', 'Store_url', 'product_description',
       'Brand', 'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url_x', 'url_y', 'review_date', 'review_text'],
      dtype='object')

In [568]:
nestle_feb_raw_review['review_text'] = nestle_feb_raw_review.apply(fix_text,axis=1)
nestle_feb_raw_review.shape

(22956, 16)

In [569]:
#删除其中的'系统默认好评'
nestle_feb_raw_review= nestle_feb_raw_review[True - nestle_feb_raw_review['review_text'].str.contains('系统默认好评')]

del nestle_feb_raw_review['Store_y']
del nestle_feb_raw_review['url_y']
nestle_feb_raw_review.rename(columns={'Store_x':'Store','url_x':'url'}, inplace=True)
nestle_feb_raw_review.columns

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


Index(['Store', 'Sub_store', 'Store_url', 'product_description', 'Brand',
       'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url', 'review_date', 'review_text'],
      dtype='object')

In [570]:
nestle_feb_raw_review.shape

(22220, 14)

In [572]:
nestle_feb_raw_review.to_csv('nestle_feb_raw_review.csv', index= False, encoding='utf-8')


In [573]:
nestle_feb_raw_review['Brand'].value_counts()

Gerber    12973
NAN        6504
NAN HA     2710
MOM          33
Name: Brand, dtype: int64

### MOM Jan. and Feb.

In [574]:
local_MOM = pd.read_csv('local_MOM_nestle_jan_feb.xlsx.csv', encoding= 'utf-8')
ti_MOM = pd.read_csv('ti_MOM__jan_feb.csv', encoding= 'utf-8')

local_MOM.shape, ti_MOM.shape

((769, 15), (44, 23))

In [575]:
def get_store(url):
    if 'amazon' in url:
        return 'Amazon'
    elif 'suning' in url:
        return 'Suning'
    elif 'yhd' in url:
        return 'YHD'
    else:
        return None

ti_MOM['store'] = ti_MOM['url'].map(get_store)
ti_MOM['store'].value_counts()

Suning    44
Name: store, dtype: int64

In [576]:
ti_MOM['username'] = np.NAN
ti_MOM['variant_1'] = np.NAN
ti_MOM['variant_2'] = np.NAN
ti_MOM['review_id'] = np.NAN

# 提前修改列名
local_MOM.rename(columns={'store':'Store','retailer_product_code':'rpc','title':'product_description', 'creation_date':'review_date'},inplace=True)
ti_MOM.rename(columns={'store':'Store','retailer_product_code':'rpc','harvest_product_description':'product_description'}, inplace=True)

# 修改rpc列的数据格式, 为后面匹配做准备
mapping['rpc'] = mapping['rpc'].astype(str)
local_MOM['rpc'] = local_MOM['rpc'].astype(str)
ti_MOM['rpc'] = ti_MOM['rpc'].astype(str)

# 选取所需要的url (根据rpc, ti 的还要考虑store)
local_MOM_mapped = local_MOM.loc[local_MOM['rpc'].isin(mapping['rpc'])]
ti_MOM_mapped = ti_MOM.loc[ti_MOM['rpc'].isin(mapping['rpc']) * ti_MOM['Store'].isin(mapping['Store'])]

# 和mapping 文件做条件合并 
local_MOM_mapped_merge = pd.merge(local_MOM_mapped, mapping, on = ['rpc'])
ti_MOM_mapped_merge = pd.merge(ti_MOM_mapped, mapping, on = ['rpc'])


# 选取所需列
local_MOM_mapped_merge_out = local_MOM_mapped_merge[clm]
ti_MOM_mapped_merge_out = ti_MOM_mapped_merge[clm]

# 合并ti 和 local 
nestle_MOM_raw_review = pd.concat([ti_MOM_mapped_merge_out, local_MOM_mapped_merge_out])

nestle_MOM_raw_review['review_text'] = nestle_MOM_raw_review.apply(fix_text,axis=1)
nestle_MOM_raw_review.shape

#删除其中的'系统默认好评'
nestle_MOM_raw_review= nestle_MOM_raw_review[True - nestle_MOM_raw_review['review_text'].str.contains('系统默认好评')]

# 删除合并文件后重复的列
del nestle_MOM_raw_review['Store_y']
del nestle_MOM_raw_review['url_y']

# 修改列名, 完成所有MOM产品的合并, 清洗
nestle_MOM_raw_review.rename(columns={'Store_x':'Store','url_x':'url'}, inplace=True)
nestle_MOM_raw_review.shape



C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))
C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


(801, 14)

In [577]:
# 按照January 和February 进行数据拆分
nestle_MOM_jan_raw_review = nestle_MOM_raw_review[nestle_MOM_raw_review['review_date'].str.contains('2018-01')]
nestle_MOM_feb_raw_review = nestle_MOM_raw_review[nestle_MOM_raw_review['review_date'].str.contains('2018-02')]

In [578]:
nestle_MOM_jan_raw_review.shape, nestle_MOM_feb_raw_review.shape

((336, 14), (465, 14))

In [583]:
# 将February的所有数据合并, 下一步运POSTag java 程序
nestle_feb_raw_review_final = pd.concat([nestle_feb_raw_review, nestle_MOM_feb_raw_review])

# 写出文件
nestle_feb_raw_review_final.to_csv('nestle_feb_raw_review_final.csv', index= False, encoding= 'utf-8')

In [582]:
nestle_feb_raw_review_final['Brand'].value_counts()

Gerber    12973
NAN        6504
NAN HA     2710
MOM         498
Name: Brand, dtype: int64

In [591]:
nestle_feb_raw_review_final.columns

Index(['Store', 'Sub_store', 'Store_url', 'product_description', 'Brand',
       'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url', 'review_date', 'review_text'],
      dtype='object')

In [585]:
nestle_jan_raw_review_final = pd.concat([nestle_jan_raw_review, nestle_MOM_jan_raw_review])
nestle_jan_raw_review_final['Brand'].value_counts()

Gerber    8931
NAN       5815
NAN HA    2099
MOM        364
Name: Brand, dtype: int64

In [ ]:
nestle_MOM_jan_raw_review.to_csv('nestle_MOM_jan_raw_review.csv', index= False, encoding='utf-8')


In [586]:
nestle_jan_raw_review_final.to_csv('nestle_jan_raw_review_final.csv',index=False, encoding='utf-8')

In [588]:
nestle_MOM_jan_raw_review.to_csv('nestle_MOM_jan_raw_review.csv', index=False, encoding='utf-8')

In [590]:
mapping.to_csv('store_urls.csv', index= False, encoding='utf-8')